# Data Wrangling and Pre-processing

this file will download the csv of airline dataset and then add the longitude and latitude for each airport

In [10]:
!pip install pandas

In [11]:
import pandas as pd 

data = pd.read_csv("Airline Dataset.csv")
data

,Passenger ID,First Name,Last Name,Gender,Age,Nationality,Airport Name,Airport Country Code,Country Name,Airport Continent,Continents,Departure Date,Arrival Airport,Pilot Name,Flight Status
0,10856,Edithe,Leggis,Female,62,Japan,Coldfoot Airport,US,United States,NAM,North America,6/28/2022,CXF,Edithe Leggis,On Time
1,43872,Elwood,Catt,Male,62,Nicaragua,Kugluktuk Airport,CA,Canada,NAM,North America,12/26/2022,YCO,Elwood Catt,On Time
2,42633,Darby,Felgate,Male,67,Russia,Grenoble-Isère Airport,FR,France,EU,Europe,1/18/2022,GNB,Darby Felgate,On Time
3,78493,Dominica,Pyle,Female,71,China,Ottawa / Gatineau Airport,CA,Canada,NAM,North America,9/16/2022,YND,Dominica Pyle,Delayed
4,82072,Bay,Pencost,Male,21,China,Gillespie Field,US,United States,NAM,North America,2/25/2022,SEE,Bay Pencost,On Time
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98614,58454,Gareth,Mugford,Male,85,China,Hasvik Airport,NO,Norway,EU,Europe,12/11/2022,HAA,Gareth Mugford,Cancelled
98615,22028,Kasey,Benedict,Female,19,Russia,Ampampamena Airport,MG,Madagascar,AF,Africa,10/30/2022,IVA,Kasey Benedict,Cancelled
98616,61732,Darrin,Lucken,Male,65,Indonesia,Albacete-Los Llanos Airport,ES,Spain,EU,Europe,9/10/2022,ABC,Darrin Lucken,On Time
98617,19819,Gayle,Lievesley,Female,34,China,Gagnoa Airport,CI,Côte d'Ivoire,AF,Africa,10/26/2022,GGN,Gayle Lievesley,Cancelled


# Airport locations

reading the airports.dat file and converting it to a pandas dataframe

As the stucture has commas inside some of the fields, have to use regular expression to read it in
- import re

In [12]:
# import re
import re

# open the airports.dat file
with open("airports.dat", "r", encoding="utf-8") as file:
    lines = file.readlines()

airport_data = [re.split(r',(?=(?:[^"]*"[^"]*")*[^"]*$)', line.strip()) for line in lines]
airports_df = pd.DataFrame(airport_data, 
                           columns=[
                               "ID", 
                               "Name", 
                               "City", 
                               "Country", 
                               "IATA", 
                               "ICAO", 
                               "Latitude", 
                               "Longitude", 
                               "Elevation", 
                               "Timezone", 
                               "DST", 
                               "TzDatabase", 
                               "Type", 
                               "Source"])

# rename the columns
airports_df = airports_df.rename(columns={"Name": "Airport Name", 
                                          "Country": "Airport Country Code",
                                          "City": "Airport City"})

# convert coordinates to numeric form
airports_df["Latitude"] = pd.to_numeric(airports_df["Latitude"])
airports_df["Longitude"] = pd.to_numeric(airports_df["Longitude"])

# get the columns i need: Airport Name, Airport Country Code, Latitude, Longitude
airports_df = airports_df[["Airport Name", "Latitude", "Longitude"]]
airports_df["Airport Name"] = airports_df["Airport Name"].str.replace('"', '')
print(airports_df.shape)

airports_df.head(5)

(7698, 3)


,Airport Name,Latitude,Longitude
0,Goroka Airport,-6.081690,145.391998
1,Madang Airport,-5.207080,145.789001
2,Mount Hagen Kagamuga Airport,-5.826790,144.296005
3,Nadzab Airport,-6.569803,146.725977
4,Port Moresby Jacksons International Airport,-9.443380,147.220001


# Merging

To merge, 
- first check for unmatching airport names. 
- merge on left and right with left join


In [13]:
non_matching_airports = set(data["Airport Name"]) - set(airports_df["Airport Name"])
if non_matching_airports:
    print("Airport names in current data that dont match:")
    print(non_matching_airports)
    print(len(non_matching_airports))
else:
    print("All airport names match")

Airport names in current data that dont match:
{'Ogeranang Airport', 'Leron Plains Airport', 'Colinas Airport', 'Walter J. Koladza Airport', 'Anthony Lagoon Airport', 'Humboldt Municipal Airport', 'Itabuna Airport', 'Iaciara Airport', 'Bongo Airport', 'LBJ Ranch Airport', 'Bernay – St Martin Airport', 'Virginia Highlands Airport', 'Grand Canyon West Airport', 'Agostinho Neto Airport', 'Erdenet Airport', 'Marion County Rankin Fite Airport', 'Weam Airport', 'Kitava Airport', 'Carmelita Airport', 'Baytown Airport', 'El Charco Airport', 'Gardner Municipal Airport', 'Nowata Airport', 'Holešov Airport', 'Augustus Downs Airport', 'Leitre Airport', 'Ouadda Airport', 'Thakhek Airport', 'San Ignacio Airport', 'Torokina Airport', 'Nutwood Downs Airport', 'Carlos Ruhl Airport', 'Searle Field', 'Semporna Airport', 'Gordon Downs Airport', 'Moanda Airport', 'Sullivan County International Airport', 'Koinambe Airport', 'Frieda River Airport', 'El Refugio/La Macarena Airport', 'Marigot Seaplane Base', '

In [14]:
# merge on airport names that do match
merged_data = pd.merge(data, airports_df, on="Airport Name", how="left")
merged_data

,Passenger ID,First Name,Last Name,Gender,Age,Nationality,Airport Name,Airport Country Code,Country Name,Airport Continent,Continents,Departure Date,Arrival Airport,Pilot Name,Flight Status,Latitude,Longitude
0,10856,Edithe,Leggis,Female,62,Japan,Coldfoot Airport,US,United States,NAM,North America,6/28/2022,CXF,Edithe Leggis,On Time,NaN,NaN
1,43872,Elwood,Catt,Male,62,Nicaragua,Kugluktuk Airport,CA,Canada,NAM,North America,12/26/2022,YCO,Elwood Catt,On Time,67.816704,-115.143997
2,42633,Darby,Felgate,Male,67,Russia,Grenoble-Isère Airport,FR,France,EU,Europe,1/18/2022,GNB,Darby Felgate,On Time,45.362900,5.329370
3,78493,Dominica,Pyle,Female,71,China,Ottawa / Gatineau Airport,CA,Canada,NAM,North America,9/16/2022,YND,Dominica Pyle,Delayed,45.521702,-75.563599
4,82072,Bay,Pencost,Male,21,China,Gillespie Field,US,United States,NAM,North America,2/25/2022,SEE,Bay Pencost,On Time,32.826199,-116.972000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99438,58454,Gareth,Mugford,Male,85,China,Hasvik Airport,NO,Norway,EU,Europe,12/11/2022,HAA,Gareth Mugford,Cancelled,70.486702,22.139700
99439,22028,Kasey,Benedict,Female,19,Russia,Ampampamena Airport,MG,Madagascar,AF,Africa,10/30/2022,IVA,Kasey Benedict,Cancelled,-13.484816,48.632702
99440,61732,Darrin,Lucken,Male,65,Indonesia,Albacete-Los Llanos Airport,ES,Spain,EU,Europe,9/10/2022,ABC,Darrin Lucken,On Time,38.948502,-1.863520
99441,19819,Gayle,Lievesley,Female,34,China,Gagnoa Airport,CI,Côte d'Ivoire,AF,Africa,10/26/2022,GGN,Gayle Lievesley,Cancelled,NaN,NaN


# Cleaning Merged Data

Now that the data has been merged, there are almost 100,000 rows of data so we need to clean up any locations that dont have Longitude or Latitude

In [15]:
merged_data = merged_data.dropna(subset=["Latitude", "Longitude"])
merged_data

,Passenger ID,First Name,Last Name,Gender,Age,Nationality,Airport Name,Airport Country Code,Country Name,Airport Continent,Continents,Departure Date,Arrival Airport,Pilot Name,Flight Status,Latitude,Longitude
1,43872,Elwood,Catt,Male,62,Nicaragua,Kugluktuk Airport,CA,Canada,NAM,North America,12/26/2022,YCO,Elwood Catt,On Time,67.816704,-115.143997
2,42633,Darby,Felgate,Male,67,Russia,Grenoble-Isère Airport,FR,France,EU,Europe,1/18/2022,GNB,Darby Felgate,On Time,45.362900,5.329370
3,78493,Dominica,Pyle,Female,71,China,Ottawa / Gatineau Airport,CA,Canada,NAM,North America,9/16/2022,YND,Dominica Pyle,Delayed,45.521702,-75.563599
4,82072,Bay,Pencost,Male,21,China,Gillespie Field,US,United States,NAM,North America,2/25/2022,SEE,Bay Pencost,On Time,32.826199,-116.972000
5,39630,Lora,Durbann,Female,55,Brazil,Coronel Horácio de Mattos Airport,BR,Brazil,SAM,South America,6/10/2022,LEC,Lora Durbann,On Time,-12.482300,-41.277000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99433,70741,Olimpia,Arstall,Female,22,China,Wuzhou Changzhoudao Airport,CN,China,AS,Asia,4/23/2022,WUZ,Olimpia Arstall,On Time,23.456699,111.248001
99438,58454,Gareth,Mugford,Male,85,China,Hasvik Airport,NO,Norway,EU,Europe,12/11/2022,HAA,Gareth Mugford,Cancelled,70.486702,22.139700
99439,22028,Kasey,Benedict,Female,19,Russia,Ampampamena Airport,MG,Madagascar,AF,Africa,10/30/2022,IVA,Kasey Benedict,Cancelled,-13.484816,48.632702
99440,61732,Darrin,Lucken,Male,65,Indonesia,Albacete-Los Llanos Airport,ES,Spain,EU,Europe,9/10/2022,ABC,Darrin Lucken,On Time,38.948502,-1.863520


In [16]:
merged_data.to_json('airline.json', orient='records', lines = True)

In [17]:
airline_json = "airline.json"
df = pd.read_json(airline_json, lines=True)
csv_file = "airline.csv"
df.to_csv(csv_file, index=False)

# Airports and Freq

below will be data wrangling. 
- airports
- airports frequency
- longitude and latitudes

In [20]:
df = pd.read_csv("airline.csv")

# group df by airport name and count occurrences
airport_counts = df["Airport Name"].value_counts().reset_index()

airport_counts.columns = ['Airport Name', 'Frequency']

# create new df
new_df = airport_counts.merge(df[['Airport Name', 'Latitude', 'Longitude']].drop_duplicates(), on="Airport Name")
new_df

new_df.to_csv("airport_info.csv", index=False)